In [13]:
import ast
import pandas as pd
import numpy as np
import faiss
import os
import re
import json
from dotenv import load_dotenv
from databricks_langchain import ChatDatabricks, DatabricksEmbeddings
from langgraph.graph import StateGraph, END
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from dataclasses import dataclass, field
from typing import List, Optional

In [14]:
# Load .env for Databricks credentials
load_dotenv(override=True)
os.environ['DATABRICKS_TOKEN'] = os.getenv('DATABRICKS_TOKEN')
os.environ['DATABRICKS_HOST'] = os.getenv('DATABRICKS_HOST')

# Initialize Databricks LLM and Embedding
chat_model = ChatDatabricks(endpoint="gpt-4o-mini", temperature=0.1, max_tokens=500)  # Reduced from 2000
embedding_model = DatabricksEmbeddings(endpoint="ada-002")

# Load FAISS and metadata
df_pd = pd.read_parquet("metadata.parquet")
index = faiss.read_index("my_faiss.index")

@dataclass
class MyState:
    chat_history: List[str] = field(default_factory=list)
    query: Optional[str] = None
    core_question: Optional[str] = None
    retrieved_docs: List[str] = field(default_factory=list)
    final_response: Optional[str] = None
    keywords: List[str] = field(default_factory=list)  # <-- ADD THIS
    relevance_score: Optional[float] = None  # Optional if you're tracking relevance


In [15]:
def reformulate_query_node(state: MyState):
    # Limit chat history to last 2 turns
    state.chat_history = state.chat_history[-2:]

    hcot_query = """
    You are an expert in understanding and rephrasing user service queries based on chat history.
    Your goal is to break down the user's intent and context into a concise, reformulated question.

    Based on the chat history: {chat_history}
    Current query: {input}

    Reformulated question:
    """
    prompt = PromptTemplate(template=hcot_query, input_variables=["chat_history", "input"])
    core_question_chain = prompt | chat_model
    core_question = core_question_chain.invoke({
        "chat_history": "\n".join(state.chat_history),
        "input": state.query
    }).content
    state.core_question = core_question
    return state


### Let's **improve** the parser to handle this more **robustly**, even with weird formatting:


### **Updated `extract_response_from_string` Function**:

def extract_response_from_string(json_string):
    json_content = json_string.strip().strip('```').replace('json', '').strip()
    try:
        json_pattern = re.compile(r'{.*}', re.DOTALL)
        match = json_pattern.search(json_content)
        if match:
            json_obj = json.loads(match.group())
            return json_obj.get("response", "No valid response found.")
        else:
            print("No JSON block found in text.")
            return "No JSON found in response."
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        return "Failed to parse JSON."



def retrieve_chunks_node(state: MyState):
    query_vector = embedding_model.embed_query(state.core_question)
    query_vector = np.array(query_vector).astype("float32").reshape(1, -1)
    distance, indices = index.search(query_vector, k=2)  # Limit to 2 docs
    retrieved_chunks = df_pd.iloc[indices[0]]['file_content'].tolist()
    # Truncate each retrieved chunk to max 300 characters
    truncated_chunks = [chunk[:300] for chunk in retrieved_chunks]
    state.retrieved_docs = truncated_chunks
    return state


def generate_response_node(state: MyState):
    # Limit chat history length
    state.chat_history = state.chat_history[-2:]

    # Limit total combined retrieved docs length
    max_combined_length = 1200  # Characters
    combined_docs = " ".join(state.retrieved_docs)
    if len(combined_docs) > max_combined_length:
        combined_docs = combined_docs[:max_combined_length] + "..."

    user_query = state.core_question + "\n" + combined_docs

    print("=" * 30)
    print(f"Prompt length: {len(user_query)} characters")
    print("=" * 30)

    prompt_prefix = """
Respond strictly in JSON format.

Use this structure:
{{
  "response": "answer with inline citations after each fact",
  "confidence": "a number from 1 to 10",
  "reason": "brief reason for your confidence"
}}

Context:
####{user_query}####
"""

    final_prompt = PromptTemplate(template=prompt_prefix, input_variables=["user_query"])
    final_response_chain = final_prompt | chat_model
    final_response = final_response_chain.invoke({"user_query": user_query})

    # Handle None case
    if final_response is None or final_response.content is None:
        print("⚠️ LLM returned None.")
        state.final_response = '{"response": "LLM did not return any content.", "confidence": 0, "reason": "No output."}'
    else:
        state.final_response = final_response.content

    # Print raw output
    print("=" * 30)
    print("RAW LLM Response:")
    print(state.final_response)
    print("=" * 30)

    # Update chat history
    response_text = extract_response_from_string(state.final_response)
    state.chat_history.append(f"User: {state.query}")
    state.chat_history.append(f"Assistant: {response_text}")
    return state



def extract_keywords_node(state: MyState):
    keyword_prompt = """
    Extract important keywords from this query for retrieval: {query}
    Only list the keywords separated by commas.
    """
    prompt = PromptTemplate(template=keyword_prompt, input_variables=["query"])
    keyword_chain = prompt | chat_model
    keywords = keyword_chain.invoke({"query": state.query}).content
    state.keywords = [k.strip() for k in keywords.split(",")]
    return state


def keyword_search_node(state: MyState):
    keyword_docs = []
    for keyword in state.keywords:
        matches = df_pd[df_pd['file_content'].str.contains(keyword, case=False, na=False)]
        keyword_docs.extend(matches['file_content'].tolist())

    # Limit keyword results and truncate
    keyword_docs = [doc[:300] for doc in keyword_docs[:2]]  # Top 2 keyword matches, each 300 chars

    # Combine with FAISS retrieved docs
    combined_docs = list(set(state.retrieved_docs + keyword_docs))[:4]  # Max 4 combined docs
    state.retrieved_docs = combined_docs
    return state

def evaluate_relevance_node(state: MyState):
    query_vector = embedding_model.embed_query(state.core_question)
    doc_vectors = [embedding_model.embed_query(doc) for doc in state.retrieved_docs]
    similarities = [np.dot(query_vector, doc_vec) / (np.linalg.norm(query_vector) * np.linalg.norm(doc_vec)) for doc_vec in doc_vectors]
    avg_similarity = np.mean(similarities)
    state.relevance_score = avg_similarity  # Optional for reporting
    return state


In [16]:
# LangGraph: Define Graph and Flow
workflow = StateGraph(MyState)

workflow.add_node("extract_keywords", RunnableLambda(extract_keywords_node))
workflow.add_node("reformulate_query", RunnableLambda(reformulate_query_node))
workflow.add_node("retrieve_chunks", RunnableLambda(retrieve_chunks_node))
workflow.add_node("keyword_search", RunnableLambda(keyword_search_node))
workflow.add_node("generate_response", RunnableLambda(generate_response_node))
workflow.add_node("evaluate_relevance", RunnableLambda(evaluate_relevance_node))

workflow.set_entry_point("extract_keywords")
workflow.add_edge("extract_keywords", "reformulate_query")
workflow.add_edge("reformulate_query", "retrieve_chunks")
workflow.add_edge("retrieve_chunks", "keyword_search")
workflow.add_edge("keyword_search", "generate_response")
workflow.add_edge("generate_response", "evaluate_relevance")
workflow.add_edge("evaluate_relevance", END)

graph = workflow.compile()


In [17]:
# Example usage:
# Set initial user query
initial_state = MyState(
    chat_history=[],
    query="Do you know of a method, which does not involve a total rebuild, for changing the ‘ExpSQLSvc’ and ‘ExpSQLAgtSvc’ accounts from local to domain users, please?"
)
# Run the graph
final_state = graph.invoke(initial_state)

# Print the final response
print(extract_response_from_string(final_state["final_response"]))


Prompt length: 1338 characters
RAW LLM Response:
```json
{
  "response": "Yes, it is possible to change the 'ExpSQLSvc' and 'ExpSQLAgtSvc' accounts from local to domain users without performing a complete rebuild. This can typically be done through the service properties in the Windows Services management console, where you can specify the account under which the service runs. However, it is important to ensure that the domain accounts have the necessary permissions and rights to run the services effectively (Honeywell, 2010). Additionally, proper configuration and testing should be conducted to avoid service disruptions (Honeywell, 2019).",
  "confidence": 8,
  "reason": "The information is based on standard practices for Windows services and the context provided suggests familiarity with service management."
}
```
Yes, it is possible to change the 'ExpSQLSvc' and 'ExpSQLAgtSvc' accounts from local to domain users without performing a complete rebuild. This can typically be done throu